# Documentation

The `DataAnalyzer` class conducts thorough analysis of flight trajectory data by systematically applying the `AngleAnalyzer` class to every single trackpoint. This process enables the determination of whether each point lies on a straight line or a curve. It reads flight data from a CSV file, processes it by extracting past and future coordinates for each point, and performs angle-based and linear regression analyses. The analysis results, including the classification of each point as belonging to a straight-line segment or a curved segment, are appended to the dataset. Finally, the processed data, enriched with analysis outcomes, is exported to a new CSV file. This systematic approach empowers the identification of different trajectory characteristics throughout the flight path.

Input:
- `.csv` file of format (processed using `FileConverter`) `timestamp [UTC],relative altitude [m],horizontal velocity [m/s],vertical velocity [m/s],distance to takeoff [km],longitude,latitude`

Output:
- `.csv` file of format `timestamp [UTC],relative altitude [m],horizontal velocity [m/s],vertical velocity [m/s],distance to takeoff [km],longitude,latitude,status,position_str,position_int,average_r_value,average_p_value,average_std_err`

# Imports

In [ ]:
import os
import sys
import pandas as pd

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import helpers.data_analyzer as dataanalyzer
import algorithms.angle_analyzer as angleanalyzer
import helpers.data_visualizer as datavisualizer

# Variables

In [ ]:
CSV_FILE_IN: str = "/Users/nicolas/Dropbox/3_Bildung/2 KZO/Stichwortverzeichnis/0 Maturarbeit/6 Wettbewerbe/1_SJf/3_Coaching/2_Projekt/software/flight-analyzer/docs/datasets/tracklogs/2_normalized/20211016_tracklog-normalized_nicolas-huber.csv"

# Initialisation

In [ ]:
DataAnalyzer: dataanalyzer.DataAnalyzer = dataanalyzer.DataAnalyzer(csv_file_in=CSV_FILE_IN)
Visualizer: datavisualizer.DataVisualizer = datavisualizer.DataVisualizer()

# Data Import

In [ ]:
data: pd.DataFrame = DataAnalyzer.read_csv_data()

# Data Processing

In [ ]:
AngleAnalyzer: angleanalyzer.AngleAnalyzer = DataAnalyzer.construct_angle_analyzer()
data_processed: pd.DataFrame = DataAnalyzer.process_data(data=data, AngleAnalyzer=AngleAnalyzer)

# Visualisation

In [ ]:
Visualizer.visualize_points_colored(df=data_processed)
Visualizer.visualize_points_position(data=data_processed)

# Export Data

In [ ]:
DataAnalyzer.export_to_csv(data_processed=data_processed)

# Report

In [ ]:
print(f"You lost {len(data)-len(data_processed)} rows of data due to processing. The data loss is supposed to be {constants.ANGLE_FUTURE_THRESHOLD + constants.ANGLE_PAST_THRESHOLD} rows, which can be calculated by adding the ANGLE_FUTURE_THRESHOLD and the ANGLE_PAST_TRESHOLD.")
print()
print(f"The average accuracy of the AngleAnalyzer algorithm and the past / future tresholds of {constants.ANGLE_PAST_THRESHOLD} / {constants.ANGLE_FUTURE_THRESHOLD} for points on a straight line can be defined as follows:")
print(f"--> average r_value: {round(data_processed[data_processed['position_int'] == 0]['average_r_value'].mean(), 2)}")
print(f"--> average p_value: {round(data_processed[data_processed['position_int'] == 0]['average_p_value'].mean(), 2)}")
print(f"--> average std_err: {round(data_processed[data_processed['position_int'] == 0]['average_std_err'].mean(), 2)}")
print()
print("A linear regression can be considered as a good fit if the r_value is close to 1, the p_value is close to 0 and the std_err is close to 0.")
print()
print(f"The system found {(data_processed['position_int'] == 0).sum()} points on straight lines, whereas the amount of points on a curve is {(data_processed['position_int'] == 1).sum()}. The expected amount of points on a curve is {len(data_processed) - (data_processed['position_int'] == 0).sum()}, which can be calculated by subtracting the count of points on a straight line from the total point count.")
print()
print(f"In total, you lost {round((data_processed["position_int"] == 1).sum() / len(data_processed) * 100, 2)}% of the data after applying the AngleAnalyzer algorithm as you can only use the points on a straight line for further processing.")

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")